In [ ]:
import time
import ccxt
import pandas as pd
from datetime import datetime, timezone

# setup exchange
bin = ccxt.binance()

# define paiar
pairs = ["USDC/USDT", "ETH/USDC", "PEPE/USDC"]
timeframe = "1m"
# set interval, in our case it's one year 01.05.24-01.05.25
since_start = int(datetime(2024, 5, 1, tzinfo=timezone.utc).timestamp() * 1000)
all_dfs = []

# it's not allowed to download more than 1000 points per one request, that's why
# fetch data in chunks per pair
for pair in pairs:
    since = since_start
    print(f"Fetching data for {pair}...")
    ohlcv_data = []
    while True:
        try:
            ohlcv = bin.fetch_ohlcv(pair, timeframe, since=since, limit=1000)
            if not ohlcv:
                break
            ohlcv_data += ohlcv
            # move further
            since = ohlcv[-1][0] + 60000  # 1m = 60000ms
            time.sleep(0.1)
        except (ccxt.NetworkError, ccxt.ExchangeError) as e:
            print(f"Error fetching {pair}: {e}")
            break

    # convert to DataFrame and tag with pair name
    df = pd.DataFrame(ohlcv_data, columns=["timestamp", "open", "high", "low", "close", "volume"])
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")
    df["pair"] = pair
    all_dfs.append(df)

# combine all pairs into one DataFrame
df_all = pd.concat(all_dfs).reset_index(drop=True)
print(df_all.tail())


Fetching data for USDC/USDT...
Fetching data for ETH/USDC...
Fetching data for PEPE/USDC...
                  timestamp      open      high       low     close  \
1626390 2025-05-12 11:30:00  0.000015  0.000015  0.000015  0.000015   
1626391 2025-05-12 11:31:00  0.000015  0.000015  0.000015  0.000015   
1626392 2025-05-12 11:32:00  0.000015  0.000015  0.000015  0.000015   
1626393 2025-05-12 11:33:00  0.000015  0.000015  0.000015  0.000015   
1626394 2025-05-12 11:34:00  0.000015  0.000015  0.000015  0.000015   

               volume       pair  
1626390  9.742904e+08  PEPE/USDC  
1626391  2.962332e+09  PEPE/USDC  
1626392  5.107066e+09  PEPE/USDC  
1626393  1.668323e+09  PEPE/USDC  
1626394  1.585380e+09  PEPE/USDC  


In [41]:
df_all

,timestamp,open,high,low,close,volume,pair
0,2024-05-01 00:00:00,1.000800,1.000900,1.000800,1.000900,9.794200e+04,USDC/USDT
1,2024-05-01 00:01:00,1.000900,1.000900,1.000800,1.000800,1.026190e+05,USDC/USDT
2,2024-05-01 00:02:00,1.000800,1.000900,1.000800,1.000800,2.994160e+05,USDC/USDT
3,2024-05-01 00:03:00,1.000900,1.000900,1.000800,1.000900,1.028580e+05,USDC/USDT
4,2024-05-01 00:04:00,1.000800,1.000900,1.000800,1.000900,9.048650e+05,USDC/USDT
...,...,...,...,...,...,...,...
1626390,2025-05-12 11:30:00,0.000015,0.000015,0.000015,0.000015,9.742904e+08,PEPE/USDC
1626391,2025-05-12 11:31:00,0.000015,0.000015,0.000015,0.000015,2.962332e+09,PEPE/USDC
1626392,2025-05-12 11:32:00,0.000015,0.000015,0.000015,0.000015,5.107066e+09,PEPE/USDC
1626393,2025-05-12 11:33:00,0.000015,0.000015,0.000015,0.000015,1.668323e+09,PEPE/USDC


In [ ]:
# split into datasets per pair for further analysis
df_eth = df_all[df_all["pair"] == "ETH/USDC"].reset_index(drop=True)
df_usdt = df_all[df_all["pair"] == "USDC/USDT"].reset_index(drop=True)
df_pepe = df_all[df_all["pair"] == "PEPE/USDC"].reset_index(drop=True)

In [ ]:
# save data
df_eth.to_csv("ETH_USDC_1m_from_01-05-2024.csv", index=False)
df_usdt.to_csv("USDC_USDT_1m_from_01-05-2024.csv", index=False)
df_pepe.to_csv("PEPE_USDC_1m_from_01-05-2024.csv", index=False)